# Bài tập 1: Multinomial Orders

Sử dụng thư viện

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/food_order.csv')

print(df.head())

         cuisine     city       device user_segment    promo fav_ingredient  \
0       japanese  city_19          web      student  promo_8         sesame   
1        chinese   city_1          ios       budget  promo_5            soy   
2  mediterranean  city_14  desktop_app       family  promo_8         yogurt   
3        italian  city_15       tablet          new  promo_4          lemon   
4     vietnamese  city_10          ios          new  promo_8          basil   

  spice_level day_of_week price_bucket  
0      medium         Tue          mid  
1   extra_hot         Wed    very_high  
2        none         Mon          mid  
3        none         Sat          mid  
4        mild         Tue          mid  


In [3]:
Train_size = round(len(df)*0.7)
target_feature = 'cuisine'
X_Train = df[:Train_size].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Train = df[target_feature][:Train_size]

X_Test = df[Train_size:].drop(target_feature, axis = 1).reset_index(drop=True, inplace=False)
Y_Test = df[target_feature][Train_size:].reset_index(drop=True, inplace=False)

In [4]:
for col in X_Train.columns:
  le_encoder = LabelEncoder()
  X_Train[col] = le_encoder.fit_transform(X_Train[col])
  X_Test[col] = le_encoder.fit_transform(X_Test[col])

clf_1 = MultinomialNB()
clf_1.fit(X_Train, Y_Train)
y_pred = clf_1.predict(X_Test)
print(accuracy_score(y_pred, Y_Test))


0.17666666666666667


# Động đất Nepal

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.preprocessing import LabelEncoder

Đọc dữ liệu từ file

In [3]:
df_train = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/train_values.csv')
df_labels = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/train_labels.csv')
df_test = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/test_values.csv')
df_test_labels = pd.read_csv('/home/vitquay1708/Study_Space/ml/week2/data/submission_format.csv')

print(df_test_labels.iloc[:, 1].value_counts())

damage_grade
1    86868
Name: count, dtype: int64


## Bài 1:

Lấy ra các dữ liệu dạng category

In [4]:
X_category_train = df_train[['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']]
y_category_train = df_labels['damage_grade']
y_category_test = df_test_labels['damage_grade']
X_category_test = df_test[['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']]


print(X_category_train)

       land_surface_condition foundation_type roof_type ground_floor_type  \
0                           t               r         n                 f   
1                           o               r         n                 x   
2                           t               r         n                 f   
3                           t               r         n                 f   
4                           t               r         n                 f   
...                       ...             ...       ...               ...   
260596                      n               r         n                 f   
260597                      t               r         n                 f   
260598                      t               r         q                 f   
260599                      t               r         x                 v   
260600                      n               r         n                 f   

       other_floor_type position plan_configuration legal_ownership_status 

Tính xác suất Pk = Nk/N

In [5]:
labels = np.zeros((len(set(y_category_train)), 2), dtype=np.float64) 

id = 0
value_count = y_category_train.value_counts()
id = list(value_count.index)
index = 0
for point in set(y_category_train):
  labels[index, 0] = point
  labels[index, 1] = value_count[value_count.index == point]/len(y_category_train)
  index+=1
print(labels)



[[1.         0.09640792]
 [2.         0.56891186]
 [3.         0.33468022]]


/tmp/ipykernel_8571/835561826.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  labels[index, 1] = value_count[value_count.index == point]/len(y_category_train)


In [6]:
num_labels_M = np.array([X_category_train.iloc[:, j].nunique() for j in range(X_category_train.shape[1])])
print(num_labels_M)

[ 3  5  3  5  4  4 10  4]


Tính xác suất P(xi|y = k) 

In [7]:
def train(X, y, alpha = 1.0):
  classes = np.unique(y) # Các lớp
  num_features = X.shape[1] # số lượng feature

  probs_class = {} #Xác suất tiên nghiệm với key là class, values là xác suất
  probs_pxik = {} # Xác suất có điều kiện của từng giá trị trong từng feature trong mỗi class c với key là class, value là danh sách cho từng feature

  for c in classes:
    Xc = X[y == c]
    probs_class[c] = len(Xc) / len(y) # tính toán xác suất tiên nghiệm
    probs_pxik[c] = [] #Mỗi phần tử của key c sẽ là một dictionary, tương ứng với từng feature
    for j in range(num_features):
      values_features, count = np.unique(Xc.iloc[:, j],return_counts = True)
      num_discrete_value = num_labels_M[j]
      probs_j = {} #key là từng giá trị trong feature đó, values là xác suất
      total = len(Xc)

      #Sử dụng laplace smoothing để tính xác suất pxik
      for v, cnt in zip(values_features, count):
        probs_j[v] = (cnt + alpha) / (total + num_discrete_value * alpha)

      for v in np.unique(X.iloc[:, j]):
        if v not in probs_j:
          probs_j[v] = alpha / (total + num_discrete_value * alpha) 
      probs_pxik[c].append(probs_j)
  return classes, probs_class, probs_pxik

In [8]:
def predict_nb(X_input, classes, probs_class, probs_pxik):
  log_probs = {c: np.log(probs_class[c]) for c in classes}

  for c in classes:
    for j, xj in enumerate(X_input):
      prob = probs_pxik[c][j].get(xj)
      log_probs[c] += np.log(prob)
  return max(log_probs, key = log_probs.get)



In [12]:
classes, probs_class, probs_pxik = train(X_category_train, y_category_train)

y_pred = np.zeros(len(y_category_test), dtype = np.float64) 

for i in range(len(X_category_test)):
  y_pred[i] = predict_nb(X_category_test.iloc[i, :], classes, probs_class, probs_pxik)
  if i % 50 == 0:
    print(i)


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [13]:
print(accuracy_score(y_pred, y_category_test))

0.0876156927752452


b) Sử dụng thư viện sklearn

In [15]:
for col in X_category_train.select_dtypes(include=['object', 'category']).columns:
  encoder = LabelEncoder()
  X_category_train.loc[:, col] = encoder.fit_transform(X_category_train.loc[:, col])
  X_category_test.loc[:, col] = encoder.transform(X_category_test.loc[:, col])

clf1 = MultinomialNB()
clf1.fit(X_category_train, y_category_train)
y_pred = clf1.predict(X_category_test)

print(accuracy_score(y_pred, y_category_test))


0.07749689183588894


## Bài 2

Lấy ra các trường dữ liệu có dạng binary

In [17]:
x_train_binary = df_train[['has_superstructure_adobe_mud',
                           'has_superstructure_mud_mortar_stone',
                           'has_superstructure_stone_flag',
                           'has_superstructure_cement_mortar_stone',
                           'has_superstructure_mud_mortar_brick',
                           'has_superstructure_cement_mortar_brick',
                           'has_superstructure_timber',
                           'has_superstructure_bamboo',
                           'has_superstructure_rc_non_engineered',
                           'has_superstructure_rc_engineered',
                           'has_superstructure_other',
                           'has_secondary_use',
                           'has_secondary_use_agriculture',
                           'has_secondary_use_hotel',
                           'has_secondary_use_rental',
                           'has_secondary_use_institution',
                           'has_secondary_use_school',
                           'has_secondary_use_industry',
                           'has_secondary_use_health_post',
                           'has_secondary_use_gov_office',
                           'has_secondary_use_use_police',
                           'has_secondary_use_other']]

x_test_binary = df_test[['has_superstructure_adobe_mud',
                           'has_superstructure_mud_mortar_stone',
                           'has_superstructure_stone_flag',
                           'has_superstructure_cement_mortar_stone',
                           'has_superstructure_mud_mortar_brick',
                           'has_superstructure_cement_mortar_brick',
                           'has_superstructure_timber',
                           'has_superstructure_bamboo',
                           'has_superstructure_rc_non_engineered',
                           'has_superstructure_rc_engineered',
                           'has_superstructure_other',
                           'has_secondary_use',
                           'has_secondary_use_agriculture',
                           'has_secondary_use_hotel',
                           'has_secondary_use_rental',
                           'has_secondary_use_institution',
                           'has_secondary_use_school',
                           'has_secondary_use_industry',
                           'has_secondary_use_health_post',
                           'has_secondary_use_gov_office',
                           'has_secondary_use_use_police',
                           'has_secondary_use_other']]
y_train_binary = df_labels['damage_grade']
y_test_binary = df_test_labels['damage_grade']

print(x_train_binary.shape)
print(x_test_binary.shape)
print(y_train_binary.shape)
print(y_test_binary.shape)

(260601, 22)
(86868, 22)
(260601,)
(86868,)


In [18]:
num_labels_M = np.array([x_train_binary.iloc[:, j].nunique() for j in range(x_train_binary.shape[1])])
print(num_labels_M)


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [19]:
classes_b, probs_class_b, probs_pxik_b = train(x_train_binary, y_train_binary)

y_pred_b = np.zeros(len(y_test_binary), dtype = np.float64) 

for i in range(len(x_test_binary)):
  y_pred_b[i] = predict_nb(x_test_binary.iloc[i, :], classes_b, probs_class_b, probs_pxik_b)
  if i % 50 == 0:
    print(i)


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [23]:
accuracy_score_b = accuracy_score(y_pred_b, y_test_binary)
print(accuracy_score_b)

0.09378597412165585


b) Sử dụng thư viện sklearn

In [22]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(x_train_binary, y_train_binary)
y_pred_b_lib = clf.predict(x_test_binary)

print(accuracy_score(y_pred_b_lib, y_test_binary))

0.09378597412165585
